In [1]:
from pymongo import MongoClient
import random
import numpy as np
def retrieveRandomFromMongo(num_entries, agentID, lossFn, query=None):
    client = MongoClient("mongodb://localhost:27017/")
    db = client['sequentialdata']
    collection = db[f'mode_20_Q_Agent{agentID}'] 
    
    if query is None:
        query = {}

    all_documents = list(collection.find(query))
    if num_entries >= len(all_documents):
        # If num_entries is greater than or equal to the total number of documents, return all documents
        retDocs =  all_documents

    else:
        # Randomly select the required number of entries
        sampled_documents = random.sample(all_documents, num_entries)
        retDocs =  sampled_documents

    outputSamples = []
    if lossFn == "CE" or lossFn == "L1":
        for doc in retDocs:
            x = np.array(doc["obs"], dtype=np.float32)
            prev_actions = np.array(one_hot_max_with_tie_resolution(np.array(doc["prev_qvals"], dtype=np.float32)), dtype=np.float32)
            outputSamples.append((x, prev_actions))

    elif lossFn == "MSE":
        for doc in retDocs:
            x = np.array(doc["obs"], dtype=np.float32)
            prev_actions = np.array(doc["prev_qvals"], dtype=np.float32)
            outputSamples.append((x, prev_actions))

    return outputSamples

In [2]:
samples = retrieveRandomFromMongo(100, 1, "MSE")

In [25]:
batch_size = 50
import torch
import torch.nn as nn
import torch.optim as optim
data_loader = torch.utils.data.DataLoader(samples,
                                              batch_size=batch_size,
                                              shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
inputs_list = []
labels_list = []

# Load data from DataLoader into Python lists or tensors
for sample in samples:
    inputs_list.append
    data in data_loader:
    inputs, labels = data[0], data[1]
    inputs_list.append(inputs)
    labels_list.append(labels)

# Convert the lists into tensors or arrays
inputs_tensor = torch.cat(inputs_list, dim=0)  # Concatenate along the batch dimension
labels_tensor = torch.cat(labels_list, dim=0)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    inputs_tensor, labels_tensor, test_size=0.2, random_state=42
)

In [16]:
X_train[0]

tensor([0.3333, 0.0000, 0.3333, 0.6667, 0.3333, 0.6667, 0.3333, 0.7778, 0.0000,
        1.0000])

In [17]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
  def __init__(self, X_train, y_train):
    self.X = X_train
    self.y = y_train
    self.len = self.X.shape[0]
  
  def __getitem__(self, index):
    return self.X[index], self.y[index]
  def __len__(self):
    return self.len

In [18]:
traindata = Data(X_train, y_train)

In [26]:
trainloader = DataLoader(traindata, batch_size=batch_size, 
                         shuffle=True, num_workers=2)

In [28]:
import torch.nn as nn
input_dim = 10
# number of hidden layers
hidden_layers = 25
# number of classes (unique of y)
output_dim = 5

class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.linear1 = nn.Linear(input_dim, hidden_layers)
    self.linear2 = nn.Linear(hidden_layers, output_dim)
  def forward(self, x):
    x = self.linear1(x)
    x = self.linear2(x)
    return x

In [31]:
clf = Network()
print(clf.parameters)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(clf.parameters(), lr=0.1)

<bound method Module.parameters of Network(
  (linear1): Linear(in_features=10, out_features=25, bias=True)
  (linear2): Linear(in_features=25, out_features=5, bias=True)
)>


TypeError: 'DataLoader' object is not subscriptable

In [35]:
epochs = 2
for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
      print(data)
      
   

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/athmajanvivekananthan/miniconda3/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/athmajanvivekananthan/miniconda3/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/athmajanvivekananthan/miniconda3/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/athmajanvivekananthan/miniconda3/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'Data' on <module '__main__' (built-in)>
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^

RuntimeError: DataLoader worker (pid(s) 49978, 49979) exited unexpectedly